# ============================================================
# 🐶 YOLOv8 - Detector de Cachorro
# Autor: Saulo Pereira
# Projeto: Dog Detector (Dataset da cachorra)
# ============================================================

# === 1. Instalar dependências ===

In [7]:
# ...existing code...
import sys
print("Python:", sys.version.splitlines()[0])
if sys.version_info.major == 3 and sys.version_info.minor >= 13:
    raise SystemExit("AVISO: Python 3.13 pode exigir compilador para numpy. Use Conda/Python 3.11 ou instale Visual Studio Build Tools (winget install --id Microsoft.VisualStudio.2022.BuildTools -e) antes de rodar 'pip install' no notebook.")
# Se estiver em Python compatível (3.11/3.10), você pode descomentar a linha abaixo para instalar:
# !pip -q install ultralytics==8.2.103 opencv-python pandas matplotlib tqdm torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121
# ...existing code...

Python: 3.10.19 (main, Oct 21 2025, 16:43:05) [GCC 11.2.0]


In [8]:

!pip -q install ultralytics==8.2.103 opencv-python pandas matplotlib tqdm torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

# === 2. Importar bibliotecas ===

In [9]:

from ultralytics import YOLO
import torch, os, glob, csv, cv2, pandas as pd
print("Torch:", torch.__version__, "| CUDA disponível:", torch.cuda.is_available())

Torch: 2.5.1+cu124 | CUDA disponível: True


# === 3. Verificar estrutura do dataset ===

In [10]:
!tree datasets/doggo -L 2 || ls datasets/dog

/bin/bash: line 1: tree: command not found
ls: cannot access 'datasets/dog': No such file or directory


# === 4. Ler arquivo dataset.yaml ===

In [11]:
print("==== dataset.yaml ====")
print(open('../dataset.yaml', 'r').read())

==== dataset.yaml ====
path: /mnt/c/Users/Aluno/Downloads/dog_detector_scaffold/datasets/

train: dog_aug/images/train
val: dog/images/val
test: dog/images/test

names:
  0: dog



# === 5. Criar e treinar o modelo YOLOv8 ===

In [12]:
import torch
print("CUDA disponível:", torch.cuda.is_available())
print("CUDA versão:", torch.version.cuda)
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "SEM GPU")


CUDA disponível: True
CUDA versão: 12.4
GPU: NVIDIA GeForce RTX 4060


In [13]:
import os, gc, torch

# Melhora fragmentação de memória CUDA
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

gc.collect()
torch.cuda.empty_cache()

from ultralytics import YOLO

# Detecta GPU automaticamente
device = 0 if torch.cuda.is_available() else 'cpu'
print("Usando dispositivo:", device)

# Carrega o modelo corretamente (sem .to())
model = YOLO('yolov8n.pt')

# Treinamento
results = model.train(
    data='../dataset.yaml',
    epochs=200,
    imgsz=800,
    batch=8,
    workers=8,
    device=device,      # usa GPU quando disponível
    amp=True,
    half=True,
    augment=True,
    single_cls=True,     # dataset com 1 classe
    lr0=0.01,
    patience=50,
    cache=True,
    project='runs_dog',
    name='det_dog_tuned',
    exist_ok=True,
    plots=True
)


Usando dispositivo: 0
New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.10.19 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=../dataset.yaml, epochs=200, time=None, patience=50, batch=8, imgsz=800, save=True, save_period=-1, cache=True, device=0, workers=8, project=runs_dog, name=det_dog_tuned, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, e

train: Scanning /mnt/c/Users/Aluno/Downloads/dog_detector_scaffold/datasets/dog_aug/labels/train... 460 images, 0 backgrounds, 0 corrupt: 100%|██████████| 460


train: New cache created: /mnt/c/Users/Aluno/Downloads/dog_detector_scaffold/datasets/dog_aug/labels/train.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.6GB RAM): 100%|██████████| 460/460 [00:10<00:00, 44.65it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

    169/200      1.62G     0.2751     0.2328     0.9075         12        800: 100%|██████████| 58/58 [00:04<00:00, 12.34it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 11.89it/s]

                   all         14         23      0.933       0.87      0.944       0.67
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 119, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



169 epochs completed in 0.243 hours.
Optimizer stripped from runs_dog/det_dog_tuned/weights/last.pt, 6.3MB
Optimizer stripped from runs_dog/det_dog_tuned/weights/best.pt, 6.3MB

Validating runs_dog/det_dog_tuned/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.10.19 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.23it/s]


                   all         14         23      0.896       0.87      0.888      0.648
Speed: 0.3ms preprocess, 8.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs_dog/det_dog_tuned


# === 6. Avaliar o modelo ===

In [14]:
metrics = model.val(
    data='../dataset.yaml',
    imgsz=640,        # pode manter
    batch=40,          # bem pequeno
    workers=6        # evita loader pesado
)
print(metrics)


Ultralytics YOLOv8.2.103 🚀 Python-3.10.19 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /mnt/c/Users/Aluno/Downloads/dog_detector_scaffold/datasets/dog/labels/val.cache... 14 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14/14 


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.0GB RAM): 100%|██████████| 14/14 [00:00<00:00, 64.95it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.96it/s]


                   all         14         23      0.717      0.913      0.889      0.639
Speed: 0.8ms preprocess, 18.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs_dog/det_dog_tuned
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f673c402ef0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    

# === 7. Fazer predições nas imagens de teste ===

In [16]:
import os
import glob
import csv
from ultralytics import YOLO

# Carrega o modelo treinado
model = YOLO('runs_dog/det_dog_tuned/weights/best.pt')

# Diretórios
input_dir  = 'datasets/dog/images/test'
output_dir = 'runs_dog/pred_test'
os.makedirs(output_dir, exist_ok=True)

rows = [("image", "dog_count")]

for img_path in sorted(glob.glob(os.path.join(input_dir, '*'))):
    res = model.predict(
        source=img_path,
        imgsz=640,
        conf=0.25,
        iou=0.5,
        save=True,
        project='runs_dog',
        name='pred_test',
        exist_ok=True
    )
    
    pred = res[0]
    boxes = pred.boxes
    
    cls = boxes.cls.cpu().numpy() if boxes is not None else []
    dog_count = int((cls == 0).sum())
    
    rows.append((os.path.basename(img_path), dog_count))

# Salvar CSV  
with open('runs_dog/pred_test_counts.csv', 'w', newline='') as f:
    csv.writer(f).writerows(rows)

print("✅ Predições salvas em runs_dog/pred_test/")
print("✅ CSV com contagem salvo em runs_dog/pred_test_counts.csv")


✅ Predições salvas em runs_dog/pred_test/
✅ CSV com contagem salvo em runs_dog/pred_test_counts.csv


# === 8. Visualizar resultados ===

In [15]:
import matplotlib.pyplot as plt
import cv2

test_img = sorted(glob.glob(os.path.join('runs_dog/pred_test', '*.jpg')))[0]
img = cv2.cvtColor(cv2.imread(test_img), cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title("Exemplo de detecção")
plt.axis("off")
plt.show()

<Figure size 640x480 with 1 Axes>